In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


# Câmara dos Deputados

## Extraindo informações


In [7]:
def extract_info_from_all_pages(base_url):
    all_data = []
    not_found_pages = []
    error_pages = []
    page = 1
    
    while True:
        print('pagina buscada:',page)
        try:
            # Monta a URL com o parâmetro da página
            url = f"{base_url}&pagina={page}"
            response = requests.get(url)
            
            # Verifica se a requisição foi bem-sucedida
            if response.status_code != 200:
                not_found_pages.append(page)
                print('erro na página')
                break
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extração das informações desejadas
            rows = soup.find_all('tr')  # Assumindo que os dados estão em uma tabela
            page_data = []
            
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 6:  # Assumindo que as colunas desejadas são Nome, CPF, Cargo, Lotação, Empresa e Contrato
                    nome = cols[0].get_text(strip=True)
                    cpf = cols[1].get_text(strip=True)
                    cargo = cols[2].get_text(strip=True)
                    lotacao = cols[3].get_text(strip=True)
                    empresa = cols[4].get_text(strip=True)
                    contrato = cols[5].get_text(strip=True)
                    pagina = page
                    
                    page_data.append([nome, cpf, cargo, lotacao, empresa, contrato, pagina])
            
            # Adiciona os dados da página atual à lista principal
            all_data.extend(page_data)
            
            # Se menos de 20 resultados foram encontrados, encerra o loop
            if len(page_data) < 20:
                break
            
            # Avança para a próxima página
            page += 1
        
        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar a página {page}: {e}")
            error_pages.append(page)
            break
    
    # Criação do DataFrame com todos os dados coletados
    df = pd.DataFrame(all_data, columns=['Nome', 'CPF', 'Cargo', 'Lotação', 'Empresa', 'Contrato', 'Pagina'])
    
    return df, not_found_pages, error_pages


In [8]:

# Exemplo de uso
base_url = 'https://www.camara.leg.br/transparencia/recursos-humanos/contratos-terceirizacao?search=&aba=terceirizados'
df, not_found_pages, error_pages = extract_info_from_all_pages(base_url)

print("DataFrame extraído:")
print(df)

if not_found_pages:
    print(f"Páginas não encontradas: {not_found_pages}")
elif error_pages:
    print(f"Páginas com erro: {error_pages}")
else:
    print("Todas as páginas foram encontradas com sucesso.")

pagina buscada: 1
pagina buscada: 2
pagina buscada: 3
pagina buscada: 4
pagina buscada: 5
pagina buscada: 6
pagina buscada: 7
pagina buscada: 8
pagina buscada: 9
pagina buscada: 10
pagina buscada: 11
pagina buscada: 12
pagina buscada: 13
pagina buscada: 14
pagina buscada: 15
pagina buscada: 16
pagina buscada: 17
pagina buscada: 18
pagina buscada: 19
pagina buscada: 20
pagina buscada: 21
pagina buscada: 22
pagina buscada: 23
pagina buscada: 24
pagina buscada: 25
pagina buscada: 26
pagina buscada: 27
pagina buscada: 28
pagina buscada: 29
pagina buscada: 30
pagina buscada: 31
pagina buscada: 32
pagina buscada: 33
pagina buscada: 34
pagina buscada: 35
pagina buscada: 36
pagina buscada: 37
pagina buscada: 38
pagina buscada: 39
pagina buscada: 40
pagina buscada: 41
pagina buscada: 42
pagina buscada: 43
pagina buscada: 44
pagina buscada: 45
pagina buscada: 46
pagina buscada: 47
pagina buscada: 48
pagina buscada: 49
pagina buscada: 50
pagina buscada: 51
pagina buscada: 52
pagina buscada: 53
pa

In [10]:
df.head()

,Nome,CPF,Cargo,Lotação,Empresa,Contrato,Pagina
0,ABELARDO BARBOSA DE SOUSA,***.730.201-**,Analista de suporte técnico,SESAU,PLANSUL PLANEJAMENTO E CONSULTORIA LTDA,211/2022,1
1,ACIVANIO DE SOUZA SANTANA,***.649.521-**,Motorista executivo,,SOLLO CONSTRUÇÕES E SERVIÇOS LTDA,161/2021,1
2,ACRISIO LOPES TABOSA,***.929.911-**,Motorista executivo,CTRAN,SOLLO CONSTRUÇÕES E SERVIÇOS LTDA,161/2021,1
3,ADAILSON SILVA DOS SANTOS,***.842.651-**,Técnico de manutenção - 5 dias,DIREX,FUNDAÇÃO PARA O DESENVOLVIMENTO DAS ARTES E DA...,78/2024,1
4,ADAILTON CARLOS SANTANA SILVA,***.621.531-**,Mecânico de instalação de split,CEQUI,PROCLIMA ENGENHARIA LTDA,39/2020,1


In [81]:
df.shape

(3737, 7)

In [13]:
df.to_csv('./terceirizados_2024_09_02.csv',index=False, header=True)

## Filtrando dados


In [73]:
# df['Cargo'].unique()

In [34]:
# Definindo a regex para filtrar cargos relacionados a TI
# regex_ti = r'(?i)\b(analista|técnico|arquiteto|supervisor|coordenador|designer|assistente)\b(?!.*audiovisuais).*(TI|infraestrutura|suporte|dados|sistemas|informática|data center)\b'
regex_ti = r'(?i)\b(analista|arquiteto|supervisor|coordenador|designer|assistente)\b(?!.*audiovisuais).*(TI|infraestrutura|suporte|dados|sistemas|informática|data center)\b'

# Aplicando o filtro usando a regex
df_ti = df[df['Cargo'].str.contains(regex_ti, na=False)]

c:\Users\gabri\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [35]:
df_ti.drop_duplicates().count()

Nome        288
CPF         288
Cargo       288
Lotação     288
Empresa     288
Contrato    288
Pagina      288
dtype: int64

In [64]:
df_ti.Cargo.value_counts()

Analista de TI nível Sênior                                    89
Analista de suporte técnico                                    88
Analista de Infraestrutura de TI Master                        26
Analista de Infraestrutura de TI Sênior                        26
Analista de Suporte ao Desenvolvimento                         17
Analista de TI - Analista de Dados - Sênior                    10
Analista de Infraestrutura de TI Pleno                          6
Analista de TI - Agile Master - sênior                          6
Analista de TI - Arquiteto de Dados - Sênior                    5
Analista de TI - Eng Dados - Sênior                             4
Analista de TI - Planejamento e Processos Digitais - Sênior     4
Analista de TI nível Master                                     4
Arquiteto de Sistemas                                           3
Name: Cargo, dtype: int64

# Senado

## Extraindo informações


In [67]:
# URL da página com a tabela
url = 'https://www6g.senado.leg.br/transparencia/licitacoes-e-contratos/contratos/terceirizados'

# Enviando uma requisição para a URL
response = requests.get(url)
response.raise_for_status()  # Garante que a requisição foi bem-sucedida

# Criando um objeto BeautifulSoup para fazer o parsing do HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Encontrando a tabela na página
table = soup.find('table')

# Inicializando listas para armazenar os dados
sequences = []
names = []
cpfs = []
functions = []
locations = []
companies = []

# Iterando sobre as linhas da tabela
for row in table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) == 6:  # Verifica se a linha tem 6 colunas
        sequences.append(cells[0].get_text(strip=True))
        names.append(cells[1].get_text(strip=True))
        cpfs.append(cells[2].get_text(strip=True))
        functions.append(cells[3].get_text(strip=True))
        locations.append(cells[4].get_text(strip=True))
        companies.append(cells[5].get_text(strip=True))

# Criando um DataFrame com os dados
data = {
    'Sequência': sequences,
    'Nome': names,
    'CPF': cpfs,
    'Função': functions,
    'Lotação': locations,
    'Empresa': companies
}
df_senado = pd.DataFrame(data)

# Salvando o DataFrame em um arquivo CSV
# df_senado.to_csv('terceirizados.csv', index=False)

# print("Tabela salva com sucesso como 'terceirizados.csv'.")

In [68]:
df_senado.shape

(3164, 6)

In [69]:
df_senado.head()

,Sequência,Nome,CPF,Função,Lotação,Empresa
0,1,Adaide José de Souza,968.***.***-04,APOIO ADMINISTRATIVO I,SECOM,R7 FACILITIES – MANUTENÇÃO E SERVIÇOS LTDA...
1,2,Adail Lima da Silva,646.***.***-49,Servente Diurno,MAISSERV,MAIS SERVIÇOS LTDA. - Contrato 4727
2,3,Adailda Ferreira Nunes,093.***.***-68,Servente Diurno,MAISSERV,MAIS SERVIÇOS LTDA. - Contrato 4727
3,4,Adailson Silva Ferreira,031.***.***-09,Vigilância Desarmada Diurna,GI,G I EMPRESA DE SEGURANÇA LTDA - Contrato 5116
4,5,Adailton Leite,573.***.***-06,Ajudante de Serviços Gerais,SINFRA,CONSTRUTORA PORTO S.A. (CONSTRUTORA PORTO) - C...


## Filtrando dados


In [80]:
# df_senado['Função'].unique()

In [77]:
# Definindo a regex para filtrar cargos relacionados a TI
# regex_ti_senado = r'(?i)\b(Técnico em Tecnologia da Informação|Desenvolvedor|Arquiteto de Plataforma|Scrum Master|Cientista de Dados|Programador|Administrador de Sistemas|Engenheiro de Dados|Desenvolvedor PL/SQL|Supervisor de Tecnologia de Dados e Pesquisa|Técnico em visualização e interação com dados em tecnologia Web)\b'
regex_ti_senado = r'(?i)\b(Desenvolvedor|Arquiteto de Plataforma|Scrum Master|Web Designer|Cientista de Dados|Programador|Engenheiro de Dados|Desenvolvedor PL/SQL|Supervisor de Tecnologia de Dados e Pesquisa)\b'
# Aplicando o filtro usando a regex
df_ti_senado = df_senado[df_senado['Função'].str.contains(regex_ti_senado, na=False)]

In [78]:
df_ti_senado.shape

(97, 6)

In [79]:
df_ti_senado['Função'].value_counts()

Programador Visual Gráfico                      24
Desenvolvedor Java Sênior                       22
Desenvolvedor Java Pleno                        20
Scrum Master                                     8
Arquiteto de Plataforma                          5
Desenvolvedor PL/SQL Sênior                      4
WEB Designer                                     4
WEB Designer - 02 Substituições                  3
Web Designer para Sistemas Corporativos          3
Engenheiro de Dados                              2
Supervisor de Tecnologia de Dados e Pesquisa     1
Cientista de Dados                               1
Name: Função, dtype: int64